In [1]:
import numpy as np
from itertools import combinations

## Lexicographic pattern

In [2]:
n_det = 24

n_comb = int((n_det * (n_det - 1) * (n_det - 2)) / 6)
crater_triads = np.empty((n_comb, 3), np.int)
for i, el in enumerate(combinations(np.arange(n_det), 3)):
    crater_triads[i] = el

crater_triads

array([[ 0,  1,  2],
       [ 0,  1,  3],
       [ 0,  1,  4],
       ...,
       [20, 21, 23],
       [20, 22, 23],
       [21, 22, 23]])

## Enhanced Pattern Shifting
Arnas, D., Fialho, M. A. A., & Mortari, D. (2017). Fast and robust kernel generators for star trackers. Acta Astronautica,
134 (August 2016), 291–302. https://doi.org/10.1016/j.actaastro.2017.02.016

In [3]:
# Verify with p. 292 from https://doi.org/10.1016/j.actaastro.2017.02.016
def enhanced_pattern_shifting(n):
    for dj in range(1, n-1):
        for dk in range(1, n-dj):
            for ii in range(1, 4):
                for i in range(ii, n-dj-dk+1, 3):
                    j = i + dj
                    k = j + dk
                    yield i, j, k

crater_triads = np.empty((n_comb, 3), np.int)
for it, (i, j, k) in enumerate(enhanced_pattern_shifting(24)):
    print((i, j, k))

(1, 2, 3)
(4, 5, 6)
(7, 8, 9)
(10, 11, 12)
(13, 14, 15)
(16, 17, 18)
(19, 20, 21)
(22, 23, 24)
(2, 3, 4)
(5, 6, 7)
(8, 9, 10)
(11, 12, 13)
(14, 15, 16)
(17, 18, 19)
(20, 21, 22)
(3, 4, 5)
(6, 7, 8)
(9, 10, 11)
(12, 13, 14)
(15, 16, 17)
(18, 19, 20)
(21, 22, 23)
(1, 2, 4)
(4, 5, 7)
(7, 8, 10)
(10, 11, 13)
(13, 14, 16)
(16, 17, 19)
(19, 20, 22)
(2, 3, 5)
(5, 6, 8)
(8, 9, 11)
(11, 12, 14)
(14, 15, 17)
(17, 18, 20)
(20, 21, 23)
(3, 4, 6)
(6, 7, 9)
(9, 10, 12)
(12, 13, 15)
(15, 16, 18)
(18, 19, 21)
(21, 22, 24)
(1, 2, 5)
(4, 5, 8)
(7, 8, 11)
(10, 11, 14)
(13, 14, 17)
(16, 17, 20)
(19, 20, 23)
(2, 3, 6)
(5, 6, 9)
(8, 9, 12)
(11, 12, 15)
(14, 15, 18)
(17, 18, 21)
(20, 21, 24)
(3, 4, 7)
(6, 7, 10)
(9, 10, 13)
(12, 13, 16)
(15, 16, 19)
(18, 19, 22)
(1, 2, 6)
(4, 5, 9)
(7, 8, 12)
(10, 11, 15)
(13, 14, 18)
(16, 17, 21)
(19, 20, 24)
(2, 3, 7)
(5, 6, 10)
(8, 9, 13)
(11, 12, 16)
(14, 15, 19)
(17, 18, 22)
(3, 4, 8)
(6, 7, 11)
(9, 10, 14)
(12, 13, 17)
(15, 16, 20)
(18, 19, 23)
(1, 2, 7)
(4, 5, 10)
(7,

In [5]:
batch_size = 30
eps_gen = enhanced_pattern_shifting(n_det)

for i in range(2):

    for index, (i, j, k) in enumerate(eps_gen):
        print(index, (i, j, k))
        if index >= batch_size -1:
            break

0 (1, 2, 3)
1 (4, 5, 6)
2 (7, 8, 9)
3 (10, 11, 12)
4 (13, 14, 15)
5 (16, 17, 18)
6 (19, 20, 21)
7 (22, 23, 24)
8 (2, 3, 4)
9 (5, 6, 7)
10 (8, 9, 10)
11 (11, 12, 13)
12 (14, 15, 16)
13 (17, 18, 19)
14 (20, 21, 22)
15 (3, 4, 5)
16 (6, 7, 8)
17 (9, 10, 11)
18 (12, 13, 14)
19 (15, 16, 17)
20 (18, 19, 20)
21 (21, 22, 23)
22 (1, 2, 4)
23 (4, 5, 7)
24 (7, 8, 10)
25 (10, 11, 13)
26 (13, 14, 16)
27 (16, 17, 19)
28 (19, 20, 22)
29 (2, 3, 5)
0 (5, 6, 8)
1 (8, 9, 11)
2 (11, 12, 14)
3 (14, 15, 17)
4 (17, 18, 20)
5 (20, 21, 23)
6 (3, 4, 6)
7 (6, 7, 9)
8 (9, 10, 12)
9 (12, 13, 15)
10 (15, 16, 18)
11 (18, 19, 21)
12 (21, 22, 24)
13 (1, 2, 5)
14 (4, 5, 8)
15 (7, 8, 11)
16 (10, 11, 14)
17 (13, 14, 17)
18 (16, 17, 20)
19 (19, 20, 23)
20 (2, 3, 6)
21 (5, 6, 9)
22 (8, 9, 12)
23 (11, 12, 15)
24 (14, 15, 18)
25 (17, 18, 21)
26 (20, 21, 24)
27 (3, 4, 7)
28 (6, 7, 10)
29 (9, 10, 13)
